<h1>Spark basics practice</h1>
<li>The code in the next cell extracts covid data from New York State's covid repository</li>
<li>The extracted data is stored in an RDD containing an Array of (String, String,Int,Int) matching (date, county, positive cases, tests) for each day since March 1st 2020 (the data is ordered by time</li>
<li>Use this RDD to answer the questions below</li>

In [1]:
//DON'T CHANGE THIS CODE!!!!

val counties = Array("Kings","Queens","New+York","Suffolk","Bronx","Nassau","Westchester","Erie",
                     "Monroe","Richmond","Onondaga","Orange","Rockland","Albany","Dutchess",
                     "Saratoga","Oneida","Niagara","Broome","Ulster","Rensselaer","Schenectady",
                     "Chautauqua","Oswego","Jefferson","Ontario","St.+Lawrence","Tompkins",
                     "Putnam","Steuben","Wayne","Chemung","Sullivan","Clinton","Cattaraugus",
                     "Cayuga","Madison","Warren","Columbia","Livingston","Washington","Herkimer",
                     "Otsego","Genesee","Fulton","Montgomery","Greene","Tioga","Franklin","Chenango",
                     "Cortland","Allegany","Delaware","Wyoming","Orleans"
                     ,"Essex","Seneca","Schoharie","Lewis","Yates","Schuyler","Hamilton")

val base_url = "https://health.data.ny.gov/resource/xdss-u53e.json?County="
val urls = counties.map(a => base_url+a) //Makes a url for each county

//This gets the contents of the url
//results is an array with one entry per county
//the data for each county is in JSON format
val results = urls.map(u => scala.io.Source.fromURL(u).mkString) 

//Create an rdd in 32 partitions (there is a lot of data)
//Reads the json and converts it into a spark dataframe (we'll do this later)
//and then converts the df into an rdd
//finally, extract the date, the county name, the new cases on that date and tests done
//on that date
val data_rdd = spark.read.json(sc.parallelize(results,32))
                    .rdd
                    .map(r => (r(5).toString.slice(0,10), 
                               r(0).toString,
                               r(4).toString.toInt,
                               r(7).toString.toInt))


Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.149:4050
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1666146196088)
SparkSession available as 'spark'


counties: Array[String] = Array(Kings, Queens, New+York, Suffolk, Bronx, Nassau, Westchester, Erie, Monroe, Richmond, Onondaga, Orange, Rockland, Albany, Dutchess, Saratoga, Oneida, Niagara, Broome, Ulster, Rensselaer, Schenectady, Chautauqua, Oswego, Jefferson, Ontario, St.+Lawrence, Tompkins, Putnam, Steuben, Wayne, Chemung, Sullivan, Clinton, Cattaraugus, Cayuga, Madison, Warren, Columbia, Livingston, Washington, Herkimer, Otsego, Genesee, Fulton, Montgomery, Greene, Tioga, Franklin, Chenango, Cortland, Allegany, Delaware, Wyoming, Orleans, Essex, Seneca, Schoharie, Lewis, Yates, Schuyler, Hamilton)
base_url: String = https://health.data.ny.gov/resource/xdss-u53e.json?County=
urls: Array[String] = Array(https://health.data.ny.gov/resource/xdss-u53e.json?County=Kings, https://health.d...


<h1>Question 1</h1>
<li>Using <span style="color:blue">reduce</span> calculate the total number of cases and total number of tests in New York State</li>

In [2]:
// val total_cases_tests = (data_rdd.reduce((a,b)=>(a._1,"New York State",a._3+b._3,a._4+b._4))._3
//                          ,data_rdd.reduce((a,b)=>(a._1,"New York State",a._3+b._3,a._4+b._4))._4)
//The return type should match with input

// val total_cases_tests = (data_rdd.map(x=>x._3).reduce(_+_),data_rdd.map(x=>x._4).reduce(_+_))
val total_cases_tests = data_rdd.map(r=> (r._3,r._4))
                                .reduce((a,b) => (a._1+b._1,a._2+b._2))
//As of 10/3/2022, the result was: (6055593,119988852)

total_cases_tests: (Int, Int) = (6113561,120799247)


<h1>Question 2</h1>
Using <span style="color:blue">reduceByKey</span> calculate the number of cases and total number of tests by county

Result as of 10/3/2022:
<pre>
cases_tests_by_county: Array[(String, (Int, Int))] = Array((Oneida,(64524,1496227)), (Tompkins,(24729,3050525)), (Chemung,(24817,469310)), (Schenectady,(40642,818993)), (Cattaraugus,(18139,301417)), (Greene,(10155,178413)), (Wyoming,(9412,154961)), (Columbia,(12685,249434)), (Chenango,(11043,231664)), (Ulster,(39314,849264)), (Clinton,(20762,398096)), (Wayne,(20377,372007)), (Herkimer,(16385,317604)), (Nassau,(498703,8184078)), (Seneca,(7049,135717)), (Lewis,(6920,102023)), (Broome,(54347,1155989)), (Erie,(252478,4154077)), (Livingston,(13481,279849)), (Bronx,(465438,8622412)), (Allegany,(10188,244807)), (Queens,(785765,14357698)), (Jefferson,(24654,394596)), (Orleans,(9907,154881)), (Putnam,(28794,477768)), (Rensselaer,(38906,859451)), (Ontario,(24127,468338)), (Suffolk,(515835,8275540...
</pre>

In [10]:
// val cases_tests_by_county = data_rdd.map(x=>(x._2,(x._3,x._4))).reduceByKey{case ((a1,a2),(b1,b2))=>(a1+b1,a2+b2)}.collect()
val cases_tests_by_county = data_rdd.map(x=>(x._2,(x._3,x._4))).reduceByKey((a,b)=>(a._1+b._1,a._2+b._2)).collect()

cases_tests_by_county: Array[(String, (Int, Int))] = Array((Oneida,(65074,1502133)), (Tompkins,(24901,3052273)), (Chemung,(25010,471440)), (Schenectady,(40929,822245)), (Cattaraugus,(18266,302432)), (Greene,(10199,178969)), (Wyoming,(9479,155606)), (Columbia,(12773,250256)), (Chenango,(11113,232787)), (Ulster,(39604,852320)), (Clinton,(20966,399367)), (Wayne,(20533,373858)), (Herkimer,(16516,319074)), (Nassau,(501298,8216981)), (Seneca,(7121,136305)), (Lewis,(6961,102392)), (Broome,(54804,1160667)), (Erie,(254007,4169002)), (Livingston,(13576,280809)), (Bronx,(467352,8656693)), (Allegany,(10228,245508)), (Queens,(789762,14416307)), (Jefferson,(24869,396765)), (Orleans,(9966,155500)), (Putnam,(28950,479805)), (Rensselaer,(39195,862334)), (Ontario,(24332,470583)), (Suffolk,(518816,8311970...


<h1>Question 3</h1>
Using <span style="color:blue">reduceByKey</span> calculate the positivity by month for New York State. Then use <span style="color:blue">Math.round</span>, <span style="color:blue">sortBy</span>, and <span style="color:blue">collect</span> to display your results as a percentage with 2 decimal places. A sample of the expected result (the percentage for 10/2022 will change depending on when you pull the data):
<pre>
(2020-03,37.89)
(2020-04,31.79)
(2020-05,5.34)
(2020-06,1.2)
(2020-07,1.08)
(2020-08,0.83)
(2020-09,0.97)
(2020-10,1.3)
(2020-11,2.98)
(2020-12,5.65)
(2021-01,6.27)
(2021-02,3.66)
(2021-03,3.35)
(2021-04,2.69)
(2021-05,1.09)
(2021-06,0.42)
(2021-07,1.56)
(2021-08,3.15)
(2021-09,2.9)
(2021-10,2.31)
(2021-11,3.55)
(2021-12,10.54)
(2022-01,15.09)
(2022-02,2.93)
(2022-03,1.95)
(2022-04,5.45)
(2022-05,7.87)
(2022-06,5.88)
(2022-07,9.54)
(2022-08,7.08)
(2022-09,6.94)
(2022-10,6.71)
</pre>

In [3]:
val cases_tests_by_month = data_rdd.map(x=>(x._1.slice(0,7),(x._3,x._4)))
                                   .reduceByKey((a,b)=>(a._1+b._1,a._2+b._2))
                                   .map(x=>(x._1,Math.round(10000*(x._2._1+0.0)/x._2._2)/100.0)).sortBy(x=>x._1)
                                   .collect()

cases_tests_by_month: Array[(String, (Int, Int))] = Array((2022-10,(64419,906506)), (2021-10,(127865,5526775)), (2020-10,(49704,3820122)), (2021-11,(172280,4849661)), (2022-01,(1236025,8190019)), (2021-01,(428453,6832476)), (2020-11,(145530,4891520)), (2022-02,(112630,3842577)), (2021-12,(842708,7992135)), (2022-03,(72052,3688789)), (2020-12,(335446,5936140)), (2021-02,(217516,5944284)), (2021-03,(230640,6894731)), (2022-04,(175453,3217515)), (2022-05,(261762,3326576)), (2021-04,(168899,6268531)), (2020-03,(83712,220934)), (2020-04,(224602,706504)), (2022-06,(161643,2750449)), (2021-05,(49362,4533456)), (2021-06,(12390,2931460)), (2022-07,(200795,2105113)), (2020-05,(63397,1186339)), (2020-06,(22368,1857871)), (2021-07,(38172,2441870)), (2022-08,(150053,2118085)), (2020-07,(21688,200032...


In [5]:
Math.round(3356.7)

res1: Long = 3357


<h1>Question 4</h1>
Return an RDD of (date,county,positivity) where positivity is the percentage of tests that are positive. For this problem, you must use the Option case class to handle the case where the divisor is zero

Result as of 10/3/2022:

<pre>
res0: Array[(String, String, Double)] = Array((2022-10-02,Kings,4.0), (2022-10-01,Kings,3.0), (2022-09-30,Kings,4.0), (2022-09-29,Kings,5.0), (2022-09-28,Kings,5.0), (2022-09-27,Kings,4.0), (2022-09-26,Kings,3.0), (2022-09-25,Kings,5.0), (2022-09-24,Kings,4.0), (2022-09-23,Kings,3.0), (2022-09-22,Kings,4.0), (2022-09-21,Kings,4.0), (2022-09-20,Kings,3.0), (2022-09-19,Kings,7.0), (2022-09-18,Kings,5.0), (2022-09-17,Kings,3.0), (2022-09-16,Kings,5.0), (2022-09-15,Kings,4.0), (2022-09-14,Kings,3.0), (2022-09-13,Kings,3.0), (2022-09-12,Kings,6.0), (2022-09-11,Kings,5.0), (2022-09-10,Kings,5.0), (2022-09-09,Ki...
</pre>

In [12]:
// val positivity = data_rdd.map{
//     case (a,b,c,0) => (a,b,0.toDouble)
//     case (a,b,c,d) => (a,b,c*100/d*1.0.toDouble)
// }.collect()

In [18]:
//when the divisor is zero, that row will not return
val positivity = data_rdd.flatMap{x=>
    try {
         Some(x._1,x._2,x._3*100/x._4*1.0.toDouble) 
    } catch {
        case e: Exception => Some(x._1,x._2,0.0)
    }
}

positivity: org.apache.spark.rdd.RDD[(String, String, Double)] = MapPartitionsRDD[41] at flatMap at <console>:25


In [8]:
positivity.collect()

res3: Array[(String, String, Double)] = Array((2022-10-17,Kings,5.0), (2022-10-16,Kings,6.0), (2022-10-15,Kings,5.0), (2022-10-14,Kings,4.0), (2022-10-13,Kings,5.0), (2022-10-12,Kings,6.0), (2022-10-11,Kings,2.0), (2022-10-10,Kings,7.0), (2022-10-09,Kings,2.0), (2022-10-08,Kings,3.0), (2022-10-07,Kings,4.0), (2022-10-06,Kings,4.0), (2022-10-05,Kings,4.0), (2022-10-04,Kings,5.0), (2022-10-03,Kings,6.0), (2022-10-02,Kings,4.0), (2022-10-01,Kings,3.0), (2022-09-30,Kings,4.0), (2022-09-29,Kings,5.0), (2022-09-28,Kings,5.0), (2022-09-27,Kings,4.0), (2022-09-26,Kings,3.0), (2022-09-25,Kings,5.0), (2022-09-24,Kings,4.0), (2022-09-23,Kings,3.0), (2022-09-22,Kings,4.0), (2022-09-21,Kings,4.0), (2022-09-20,Kings,3.0), (2022-09-19,Kings,7.0), (2022-09-18,Kings,5.0), (2022-09-17,Kings,3.0), (2022-0...


In [22]:
def divide(t: (String,String,Int,Int)): Option[(String,String,Double)] = {
    try {
        Some((t._1,t._2,100*t._3/t._4))
    } catch {
        case e: Exception => None
    }
}
val positivity = data_rdd.map(t => divide(t)).flatMap(e=>e)
positivity.collect
// data_rdd.flatMap(t => divide(t)).collect


divide: (t: (String, String, Int, Int))Option[(String, String, Double)]
positivity: org.apache.spark.rdd.RDD[(String, String, Double)] = MapPartitionsRDD[47] at flatMap at <console>:33
res14: Array[(String, String, Double)] = Array((2022-10-17,Kings,5.0), (2022-10-16,Kings,6.0), (2022-10-15,Kings,5.0), (2022-10-14,Kings,4.0), (2022-10-13,Kings,5.0), (2022-10-12,Kings,6.0), (2022-10-11,Kings,2.0), (2022-10-10,Kings,7.0), (2022-10-09,Kings,2.0), (2022-10-08,Kings,3.0), (2022-10-07,Kings,4.0), (2022-10-06,Kings,4.0), (2022-10-05,Kings,4.0), (2022-10-04,Kings,5.0), (2022-10-03,Kings,6.0), (2022-10-02,Kings,4.0), (2022-10-01,Kings,3.0), (2022-09-30,Kings,4.0), (2022-09-29,Kings,5.0), (2022-09-28,Kings,5.0), (2022-09-27,Kings,4.0), (2022-09-26,Kings,3.0), (2022-09-25,Kings,5.0), (2022-09-24,K...


In [17]:
data_rdd.flatMap(t => divide(t)).collect

22/10/18 22:39:11 WARN DAGScheduler: Broadcasting large task binary with size 15.8 MiB


res12: Array[(String, String, Double)] = Array((2022-10-17,Kings,5.0), (2022-10-16,Kings,6.0), (2022-10-15,Kings,5.0), (2022-10-14,Kings,4.0), (2022-10-13,Kings,5.0), (2022-10-12,Kings,6.0), (2022-10-11,Kings,2.0), (2022-10-10,Kings,7.0), (2022-10-09,Kings,2.0), (2022-10-08,Kings,3.0), (2022-10-07,Kings,4.0), (2022-10-06,Kings,4.0), (2022-10-05,Kings,4.0), (2022-10-04,Kings,5.0), (2022-10-03,Kings,6.0), (2022-10-02,Kings,4.0), (2022-10-01,Kings,3.0), (2022-09-30,Kings,4.0), (2022-09-29,Kings,5.0), (2022-09-28,Kings,5.0), (2022-09-27,Kings,4.0), (2022-09-26,Kings,3.0), (2022-09-25,Kings,5.0), (2022-09-24,Kings,4.0), (2022-09-23,Kings,3.0), (2022-09-22,Kings,4.0), (2022-09-21,Kings,4.0), (2022-09-20,Kings,3.0), (2022-09-19,Kings,7.0), (2022-09-18,Kings,5.0), (2022-09-17,Kings,3.0), (2022-...


<h1>Question 5</h1>
Return the tuple (date,county,positivity) where the positivity was the highest (use <span style="color:blue">takeOrdered</span>)

In [34]:
val highest_positive = positivity.takeOrdered(1)(Ordering[Double].reverse.on(x=>x._3))

highest_positive: Array[(String, String, Double)] = Array((2022-03-19,Richmond,0.0))


In [128]:
//val x = data_rdd.filter(x=>x._1=="2020-03-05" & x._2=="Nassau").collect()